# GRU+FastText 文本分类

In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU

Using TensorFlow backend.


In [2]:
MAX_NUM_WORDS = 30000
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 100

## 1. 数据预处理

### 1.1 读取训练数据

In [3]:
train = pd.read_csv('./data/train.csv')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
X_train_texts = train['comment_text'].fillna('').values
X_train_texts[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [5]:
y_train = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values
y_train[0]

array([0, 0, 0, 0, 0, 0], dtype=int64)

### 1.2 读取测试数据

In [6]:
test = pd.read_csv('./data/test.csv')
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [7]:
X_test_texts = test['comment_text'].fillna('').values
X_test_texts[0]

"Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"

### 1.3 建立token

In [8]:
token = text.Tokenizer(num_words=MAX_NUM_WORDS)
token.fit_on_texts(list(X_train_texts)+list(X_test_texts))

### 1.3 将“评论文字”转换为“数字列表”

In [9]:
X_train_seqs = token.texts_to_sequences(X_train_texts)
X_test_seqs  = token.texts_to_sequences(X_test_texts)

In [10]:
X_train_seqs[0]

[733,
 78,
 1,
 140,
 131,
 182,
 30,
 712,
 4438,
 10284,
 1252,
 86,
 368,
 51,
 2230,
 14039,
 49,
 6744,
 15,
 60,
 2624,
 151,
 7,
 2832,
 33,
 115,
 1246,
 16129,
 2517,
 5,
 50,
 59,
 256,
 1,
 370,
 31,
 1,
 46,
 29,
 144,
 72,
 3931,
 89,
 4208,
 6368,
 2687,
 1183]

### 1.4 截长补短让所有“数字列表”的长度都为100

In [11]:
X_train = sequence.pad_sequences(X_train_seqs, maxlen = MAX_SEQUENCE_LENGTH)
X_test  = sequence.pad_sequences(X_test_seqs, maxlen = MAX_SEQUENCE_LENGTH)

In [12]:
X_train[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,   733,
          78,     1,   140,   131,   182,    30,   712,  4438, 10284,
        1252,    86,   368,    51,  2230, 14039,    49,  6744,    15,
          60,  2624,   151,     7,  2832,    33,   115,  1246, 16129,
        2517,     5,    50,    59,   256,     1,   370,    31,     1,
          46,    29,   144,    72,  3931,    89,  4208,  6368,  2687,
        1183])

## 2 建立嵌入层

In [ ]:
word_index = token.word_index
print('Found {} unique tokens.'.format(len(word_index)))

In [ ]:
count = 0
for (k,v) in word_index.items():
    if count == 5:break
    print(k, v)
    count += 1

### 2.1 单词到词向量的索引

In [ ]:
import io

def load_vectors(fname):
    fine = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    vocab_and_vectors = {}
    # put words as dict indexes and vectors as words values
    for line in file:
        values = line.split()
        word = values [0].decode('utf-8')
        vector = np.asarray(values[1:], dtype='float32')
        vocab_and_vectors[word] = vector
    return vocab_and_vectors

embeddings_index = load_vectoers('cc.en.300.vec')        
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embeddings_index = load_vectoers('cc.en.300.vec')        
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embeddings_index['word']

### 2.2 单词索引到词向量的矩阵

In [ ]:
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

### 2.3 建立嵌入层

In [ ]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
keras.layers.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)

## 3. 建立GRU模型

In [13]:
model = Sequential()

In [16]:
model.add(Embedding(input_dim = MAX_NUM_WORDS,
                    output_dim = EMBEDDING_DIM,
                    input_length=MAX_SEQUENCE_LENGTH))

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
model.add(Dropout(0.2))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
from keras.layers.recurrent import GRU

In [20]:
model.add(GRU(units=300))

In [21]:
model.add(Dense(units=6, activation='sigmoid'))

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          9000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 300)               540900    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1806      
Total params: 9,542,706
Trainable params: 9,542,706
Non-trainable params: 0
_________________________________________________________________


## 4 训练模型

In [23]:
model.compile(loss='binary_crossentropy',  
              optimizer='adam', 
              metrics=['accuracy'])

In [25]:
train_history = model.fit(X_train, y_train,batch_size=32, epochs=2, verbose=2, validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 127656 samples, validate on 31915 samples
Epoch 1/2
 - 2747s - loss: 0.0582 - acc: 0.9802 - val_loss: 0.0460 - val_acc: 0.9830
Epoch 2/2
 - 3037s - loss: 0.0395 - acc: 0.9848 - val_loss: 0.0478 - val_acc: 0.9830


## 5 进行预测

In [27]:
y_pred = model.predict(X_test, batch_size=1024)

NameError: name 'submission' is not defined

In [29]:
submission = pd.read_csv('./data/sample_submission.csv')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('submission.csv', index=False)